<a href="https://colab.research.google.com/github/anushka012/Automated-Text-Scoring-Model/blob/main/question_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd

In [ ]:
# DATASET_DIR = '/content/training_set.zip'

In [ ]:
# import zipfile

# local_zip_file = '/content/training_set.zip'

# zip_refs = zipfile.ZipFile(local_zip_file, 'r')

# zip_refs.extractall('/content')
# zip_refs.close()

In [ ]:
X = pd.read_csv(( '/content/training_set_2 (1).tsv'), sep='\t', encoding='ISO-8859-1')

In [ ]:
y = X['domain1_score']

In [ ]:
X = X.drop(columns=['rater1', 'rater2'])

In [ ]:
X.head(10)

,essay_id,essay_set,essay,domain1_score
0,1,1,"In five years, my goal is to successfully obta...",8
1,2,1,My ultimate goal for the next five years is to...,7
2,3,1,A few of the goals Iâve set for myself over ...,9
3,4,1,Let me start by saying that Iâm reallyÂ exci...,8
4,5,1,From the moment I read the job description for...,6
5,6,1,In the next few years I want to get better at ...,5
6,7,1,Iâve enjoyed managing a direct report in my ...,6
7,8,1,"In my current role, Iâve been able to progre...",8
8,9,1,I looked on Google and it seems like there are...,3
9,10,1,I never understand why interviewers ask this i...,2


In [ ]:
X.tail(10)

,essay_id,essay_set,essay,domain1_score
72,76,3,My biggest weakness is probably biting off mor...,2
73,77,3,My biggest weakness is the fact that I freeze ...,0
74,78,3,My biggest weakness is probably procrastinatio...,1
75,79,3,I don't have much patience when working with a...,2
76,80,3,I struggle with organization. While it hasn't ...,3
77,81,3,"I am incredibly self-motivated, and I sometime...",4
78,82,3,"Oftentimes, I can be timid when providing cons...",5
79,83,3,"My blunt, straightforward nature has allowed m...",5
80,84,3,Public speaking makes me nervous. While I don'...,2
81,85,3,I'm not great at analyzing data or numbers. Ho...,4


In [ ]:
minimum_scores = [3, 2, 1, 4, 0, 0, 0, 0, 0]
maximum_scores = [8, 10, 9, 7, 6, 5, 9, 30, 60]

In [ ]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

In [ ]:
def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

In [ ]:
def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences


In [ ]:
def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

In [ ]:
def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [ ]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

In [ ]:
def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(81, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 81], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [ ]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

In [ ]:
cv = KFold(n_splits = 5, shuffle = True)
results = []
y_pred_list = []

count = 1

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
for traincv, testcv in cv.split(X):
    print("\n--------Fold {}--------\n".format(count))
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
            # Obtaining all sentences from the training essays.
            sentences += essay_to_sentences(essay, remove_stopwords = True)
            
    # Initializing variables for word2vec model.
    num_features = 81
    min_word_count = 1
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

    clean_train_essays = []
    
    # Generate training and testing data word vectors.
    for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=10)
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(testDataVecs)
    
    # Save any one of the 5 models.
    if count == 5:
         lstm_model.save('/content/final_lstm.h5')
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)

    count += 1


--------Fold 1--------

Training Word2Vec Model...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 81)             52812     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                37376     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 90,253
Trainable params: 90,253
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 7s 40ms/step - loss: 43.5910 - mae: 5.6303
Epoch 2/10
2/2 [==============================] - 0s 28ms/step - loss: 43.3377 - mae: 5.6142
Epoch 3/10
2/2 [==============================] - 0s 25ms/step -

In [ ]:
print("Average Kappa score after a 5-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))

Average Kappa score after a 5-fold cross validation:  0.0


In [ ]:
answer = list(input("Enter your answer:-"))

Enter your answer:- I’ve always preferred to work in groups and find that my collaborative nature is one of my strongest attributes. On projects that I directed, I work well to inspire diverse team members and work side by side with them to achieve the project goals. In fact, I’ve increased productivity by ten percent over the course of two years. 
